In [1]:
import requests
import csv
import time
from bs4 import BeautifulSoup

In [2]:
session = requests.Session()

# List to store the details of all the items
all_list = []

pages = 2

for page in range(1,10):
    
    if page == 1:
        url = "https://ikman.lk/en/ads/sri-lanka/mobile-phones/huawei?enum.condition=used&tree.brand=huawei"
    else:
        url = f"https://ikman.lk/en/ads/sri-lanka/mobile-phones/huawei?sort=date&order=desc&buy_now=0&urgent=0&page={page}&enum.condition=used&tree.brand=huawei"
        # url = url.format(page)
        
    try:
        page = session.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        # Each vehicle in a page is stored in 'li.gtm-normal-ad' class
        all_details = soup.select('li.gtm-normal-ad')
        item_size = len(all_details)

        if all_details: # check if items are in a page 
            
            for item in range(item_size): 
                # loop through each item in the page

                try:

                    # code for retrieving link for the page contains complete details of each car
                    x = all_details[item].select('a')[0]
                    href = x.get('href')
                    href = "http://ikman.lk" + href

                    # getting HTML files of each item
                    page = requests.get(href)
                    soup_1 = BeautifulSoup(page.content, 'html.parser')

                    # info dictionary to store the vehicle details
                    info = {}
                    key = ""
                    value = ""
                    
                    if soup_1.select('div.section--PpGYD > div'): # check if details are availble for an item
                                     
                        n = len(soup_1.select('div.section--PpGYD > div')[1].select('div.word-break--2nyVq'))

                        for i in range(n): 
                            # loop through the features of the item

                            if i % 2 == 0:
                                key = soup_1.select('div.section--PpGYD > div')[1].select('div.word-break--2nyVq')[i].text
                            else:
                                value = soup_1.select('div.section--PpGYD > div')[1].select('div.word-break--2nyVq')[i].text

                            if i % 2 == 1:
                                info[key.strip()] = value.strip()

                        price = soup_1.select('div.amount--3NTpl')[0].text
                        description=soup_1.select('div.description--1nRbz')[0].text
                        feature_elements = soup_1.select('div.features--3dZlt')
                        features = feature_elements[0].text if feature_elements else "No features found"
                        postedon=soup_1.select('div.subtitle-wrapper--1M5Mv')[0].text
                        if price:
                            info["price"] = price
                        else:
                            print(f"No price for the item no {item} & page {page}")
                        
                        if description:
                            info["description"] = description
                        else:
                            print(f"No description for the item no {item} & page {page}")
                        if features:
                            info["features"] = features
                        else:
                            print(f"No features for the item no {item} & page {page}")
                        if postedon:
                            info["postedon"] = postedon
                        else:
                            print(f"No postedon for the item no {item} & page {page}")
                        all_list.append(info)
                        time.sleep(2)
                    
                    else:
                        print(f"No details found for the item no {item} in page {page}")

                except requests.exceptions.RequestException as item_error:
                    print(f"Request failed: {item_error}")
                    
        else:
            print(f"No items found in page {page}")
                
                
            
    except requests.exceptions.RequestException as page_error:
            print(f"Request failed: {page_error}")
            
        
    time.sleep(2)

print(all_list)

Request failed: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Request failed: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Request failed: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Request failed: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
[{'Condition:': 'Used', 'Brand:': 'Vivo', 'Model:': 'Y85', 'Edition:': '6GB 128GB', 'price': 'Rs 8,990', 'description': '🌹🌹 𝐒𝐄𝐀𝐒𝐎𝐍𝐀𝐋 𝐎𝐅𝐅𝐄𝐑𝐒🌹🌹ORIGINAL Display | SEAL PACK10990 B grade phone 11490 A grade phone ( get free access )8900  Glass crack and Signal Issue phone RETAIL only👍ORIGINAL එකක්  අරන්  වෙනස  වටහා  ගන්න.🔥 සම්පූර්ණයෙන්ම කියවන්න, එමගින් ඔබට ගැටලු ඇති නොවේ 🔥📍 Vivo Y93  BRAND NEW CONDITION | SEAL PACK BOX 19900/- up to

In [3]:
keys = all_list[0].keys()

with open('F:/Fourth year/Research/New Folder/huwei.csv', 'w', newline='', encoding='utf-8-sig') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(all_list)